In [15]:
import torch
import torch.nn as nn

inputs = torch.tensor(
    [[0.43, 0.15, 0.89],  # Your     (x^1)
     [0.55, 0.87, 0.66],  # journey  (x^2)
     [0.57, 0.85, 0.64],  # starts   (x^3)
     [0.22, 0.58, 0.33],  # with     (x^4)
     [0.77, 0.25, 0.10],  # one      (x^5)
     [0.05, 0.80, 0.55]]  # step     (x^6)
)
d_in = inputs.shape[1]      #2
d_out = 2

batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)
batch

torch.Size([2, 6, 3])


tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

In [25]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out,
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads    #1
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        print(f"Weights: {self.out_proj.weight}")
        print(f"bias: {self.out_proj.bias}")
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        print(f"x.shape: {x.shape}")
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)         #3
        queries = self.W_query(x)    #3
        values = self.W_value(x)     #3
        print(f"keys: {keys}")

        # b 2
        # num_tokens 6
        # num_heads 2
        # head_dim 1 (d_out = 2/num_heads = 2)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)       #4
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(
            b, num_tokens, self.num_heads, self.head_dim
        )
        print(f"keys new view: {keys}")

        keys = keys.transpose(1, 2)          #5
        queries = queries.transpose(1, 2)    #5
        values = values.transpose(1, 2)      #5
        print(f"keys transposed: {keys}")

        attn_scores = queries @ keys.transpose(2, 3)   #6
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]    #7

        attn_scores.masked_fill_(mask_bool, -torch.inf)     #8

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)   #9
        print(f"context_vec: {context_vec}")
 #10
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        print(f"context_vec reshaped: {context_vec}")
        context_vec = self.out_proj(context_vec)    #11
        print(f"context_vec out_proj: {context_vec}")
        return context_vec

In [27]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

Weights: Parameter containing:
tensor([[-0.1668,  0.2270],
        [ 0.5000,  0.1317]], requires_grad=True)
bias: Parameter containing:
tensor([0.1934, 0.6825], requires_grad=True)
x.shape: torch.Size([2, 6, 3])
keys: tensor([[[-0.5740,  0.2727],
         [-0.8709,  0.1008],
         [-0.8628,  0.1060],
         [-0.4789,  0.0051],
         [-0.4744,  0.1696],
         [-0.5888, -0.0388]],

        [[-0.5740,  0.2727],
         [-0.8709,  0.1008],
         [-0.8628,  0.1060],
         [-0.4789,  0.0051],
         [-0.4744,  0.1696],
         [-0.5888, -0.0388]]], grad_fn=<UnsafeViewBackward0>)
keys new view: tensor([[[[-0.5740],
          [ 0.2727]],

         [[-0.8709],
          [ 0.1008]],

         [[-0.8628],
          [ 0.1060]],

         [[-0.4789],
          [ 0.0051]],

         [[-0.4744],
          [ 0.1696]],

         [[-0.5888],
          [-0.0388]]],


        [[[-0.5740],
          [ 0.2727]],

         [[-0.8709],
          [ 0.1008]],

         [[-0.8628],
         

In [42]:
# Excercise

torch.manual_seed(123)

# 6 tokens, embedding size 768
inputs_gpt2 = torch.randn(6, 768)

# Simulate 12 batches
batch_gpt2 = torch.stack((inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2, inputs_gpt2), dim=0)

batch_size, context_length, d_in = batch_gpt2.shape
d_out = 768
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=12)
context_vecs = mha(batch_gpt2)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

Weights: Parameter containing:
tensor([[ 0.0304, -0.0230,  0.0130,  ...,  0.0182,  0.0143,  0.0189],
        [ 0.0053,  0.0059, -0.0124,  ...,  0.0359,  0.0278, -0.0121],
        [-0.0091,  0.0349, -0.0356,  ..., -0.0181,  0.0019,  0.0209],
        ...,
        [ 0.0009,  0.0064,  0.0113,  ...,  0.0298,  0.0071,  0.0324],
        [-0.0043, -0.0153, -0.0223,  ...,  0.0044,  0.0038,  0.0215],
        [ 0.0217, -0.0117,  0.0192,  ..., -0.0099,  0.0261, -0.0286]],
       requires_grad=True)
bias: Parameter containing:
tensor([ 3.4549e-02, -2.0708e-02,  3.1787e-02,  3.4472e-02, -2.3270e-02,
         2.6689e-02, -3.0451e-02,  1.4578e-02,  2.6789e-02, -3.1229e-02,
        -3.9078e-03,  1.0054e-02, -7.7798e-04,  2.9459e-02,  1.2630e-02,
         1.5771e-02, -1.7067e-02,  5.9988e-03,  4.0226e-03, -2.5303e-02,
        -2.7038e-02,  5.7422e-03,  8.1998e-03, -3.2308e-02,  2.1858e-02,
         4.0871e-03,  3.9434e-03,  5.2643e-03,  6.1991e-03,  3.0647e-03,
        -1.3660e-02, -3.0984e-02,  2.8537e